In [1]:
# === P-TLO — Threshold por monto (Outbound Cash) ===============================
# Regla: When an Outbound Cash transaction is over [Amount] {var.Amount} CLP, apply {action}.
# Justificación: seleccionar el monto en el percentil 95 de la distribución.

import pandas as pd
import numpy as np

# -------- Parámetros editables --------
PATH = "../../data/tx_retail_core.csv"   # <-- cambia el CSV
PCTS = [90, 95, 97, 99]                   # percentiles a reportar

# -------- Carga mínima --------
df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")

# -------- Filtro según regla --------
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
mask = (
    (df["tx_direction"].astype(str).str.title() == "Outbound") &
    (df["tx_type"].astype(str).str.title() == "Cash") &
    (df["tx_base_amount"] > 0)  # montos válidos y positivos en CLP
)
g = df.loc[mask, ["tx_base_amount"]].dropna()

if g.empty:
    print("No hay transacciones elegibles para P-TLO con los filtros dados.")
else:
    s = g["tx_base_amount"].astype(float)
    stats = {f"p{p}": float(np.percentile(s, p)) for p in PCTS}

    # Recomendado: el propio p95 (entero para CLP)
    recommended_amount = int(round(stats["p95"]))

    print("=== P-TLO — Percentiles de monto (CLP, Outbound Cash) ===")
    for p in PCTS:
        v = stats[f"p{p}"]
        print(f"p{p:>2}: {v:,.0f}")
    print(f"\nAmount recomendado (p95): {recommended_amount:,.0f} CLP")


=== P-TLO — Percentiles de monto (CLP, Outbound Cash) ===
p90: 2,839,709
p95: 4,000,000
p97: 5,000,000
p99: 10,843,500

Amount recomendado (p95): 4,000,000 CLP


# Simulación alertas

In [5]:
# === P-TLO — Sensibilidad (Actual vs propuestos) ===============================
# LÓGICA EXACTA:
# tx_direction = Outbound
# AND tx_base_amount > [Amount]
# Unidad = transacciones que cumplen

import pandas as pd
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

PATH="../../data/tx_retail_whale.csv"
PARAMS={
    # "Actual":{"Amount":182_692_960},
    # "p90":   {"Amount":105_957_906},
    "p95":   {"Amount":521_000_390},
    "p97":   {"Amount":587_700_874},
    "p99":   {"Amount":941_743_242},
}

df=pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_direction"]=df["tx_direction"].astype(str).str.title()
df["tx_base_amount"]=pd.to_numeric(df["tx_base_amount"], errors="coerce")

g=df[(df["tx_direction"].eq("Outbound")) & df["tx_base_amount"].notna()].copy()
param_tbl=pd.DataFrame(PARAMS).T.rename_axis("escenario").reset_index()
print("=== P-TLO — Parámetros (Amount) ==="); display(param_tbl)

counts={k:int((g["tx_base_amount"]>v["Amount"]).sum()) for k,v in PARAMS.items()}
out=pd.DataFrame([{
    "alertas_actual":counts.get("Actual",0),
    "alertas_p90":counts.get("p90",0),
    "alertas_p95":counts.get("p95",0),
    "alertas_p97":counts.get("p97",0),
    "alertas_p99":counts.get("p99",0),
}])
print("=== P-TLO — Alertas por escenario (tx) ==="); display(out)


=== P-TLO — Parámetros (Amount) ===


,escenario,Amount
0,p95,521000390
1,p97,587700874
2,p99,941743242


=== P-TLO — Alertas por escenario (tx) ===


,alertas_actual,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,0,0,20,10,4
